In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 12]

In [2]:
path =r'Home to Work - Sheet1.csv'
testDF = pd.read_csv(path)

In [3]:
testDF.head(5)

,Left Home,"December 11, 2018 at 08:39AM",Unnamed: 2,"https://maps.google.com/?q=53.369536272938745,-6.235619199999973&z=17"
0,Arrived At Work,"December 11, 2018 at 09:34AM",NaN,"https://maps.google.com/?q=53.334460546961814,..."
1,Left Work,"December 11, 2018 at 01:18PM",NaN,"https://maps.google.com/?q=53.334068956729624,..."
2,Arrived At Work,"December 11, 2018 at 01:53PM",NaN,"https://maps.google.com/?q=53.334460546961814,..."
3,Left Work,"December 11, 2018 at 06:33PM",NaN,"https://maps.google.com/?q=53.33402489271903,-..."
4,Arrived Home,"December 11, 2018 at 07:10PM",NaN,"https://maps.google.com/?q=53.36944512097854,-..."


In [4]:
testDF.drop(testDF.columns[2:],axis=1, inplace=True)
testDF.columns = ['Location', 'Time']
testDF.head(5)

,Location,Time
0,Arrived At Work,"December 11, 2018 at 09:34AM"
1,Left Work,"December 11, 2018 at 01:18PM"
2,Arrived At Work,"December 11, 2018 at 01:53PM"
3,Left Work,"December 11, 2018 at 06:33PM"
4,Arrived Home,"December 11, 2018 at 07:10PM"


In [5]:
dateTime = "December 11, 2018 at 09:34AM"
date_str1, time_str1 = dateTime.split(" at ") # Split the text date into data and time
print(date_str1)
print(time_str1)

December 11, 2018
09:34AM


In [6]:
date_dt1 = datetime.strptime(date_str1, '%B %d, %Y') # Applying strptime to the date portion of date
print(date_dt1)

2018-12-11 00:00:00


In [7]:
dateTime = "December 11, 2018 at 09:34AM"
dateTime = dateTime.replace(" at ", ", ") # replacing the at with a comma for processing (not necessary)
print(dateTime)
datetime_dt1 = datetime.strptime(dateTime, '%B %d, %Y, %I:%M%p') # Applying strptime to the whole dat and time string
print(datetime_dt1)

December 11, 2018, 09:34AM
2018-12-11 09:34:00


In [8]:
testDF['DateTime'] = testDF['Time'].apply( lambda x: datetime.strptime(x, '%B %d, %Y at %I:%M%p')) 
testDF.drop('Time', axis=1, inplace=True)
testDF.head(5)

,Location,DateTime
0,Arrived At Work,2018-12-11 09:34:00
1,Left Work,2018-12-11 13:18:00
2,Arrived At Work,2018-12-11 13:53:00
3,Left Work,2018-12-11 18:33:00
4,Arrived Home,2018-12-11 19:10:00


In [14]:
testDF['Elapsed'] = testDF['DateTime'].diff()
testDF['Elapsed Hours'] = testDF['Elapsed'] / np.timedelta64(1, 'h')
testDF['Time'] = testDF['DateTime'].apply(lambda x: x.time())

testDF['Previous Time'] = testDF['DateTime'] - testDF['Elapsed']
testDF['Previous Time'] = testDF['Previous Time'] / np.timedelta64(1, 'h')

testDF.head(5)

TypeError: cannot perform __truediv__ with this index type: DatetimeIndex

In [ ]:
testDF = testDF.set_index('DateTime')
testDF.head(5)

In [ ]:
testDF.plot(figsize=(12,8))
plt.show()

In [ ]:
workHours = testDF[testDF['Location'] == 'Left Work']
workHours.head(10)

In [ ]:
workHours['Elapsed Hours'].plot(figsize=(12,8), kind='bar')
plt.show()

In [ ]:
arriveTime = workHours = testDF[testDF['Location'] == 'Arrived At Work']
arriveTime = arriveTime[arriveTime['Elapsed Hours'] < 5]
arriveTime = arriveTime.between_time('08:00', '11:00')
arriveTime.head()

In [ ]:
from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator

arriveTime.plot(x='Time', y='Elapsed Hours', lw=0, marker='o', figsize=(12,8))